In [76]:
import numpy as np
np.set_printoptions(threshold=np.inf, linewidth=np.inf)
import scipy.sparse as sp
from pathlib import Path
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy
import os
from tqdm import tqdm

In [77]:
#function to convert y into one-hot format for model training

def one_hot(array):
    out = np.zeros((array.size, 34))
    out[np.arange(array.size), array] = 1
    return out

BIG DATA

In [78]:
#defining file paths (PC-wont work for u)

discard_dataset_path = "D://Mahjong group project data/archive/discard_datasets"
dataset_folder_train = Path(discard_dataset_path) / "2015" 
dataset_folder_test = Path(discard_dataset_path) / "2014" 

In [79]:
#array of files
files_train = list(dataset_folder_train.iterdir())[:1000]
files_test = list(dataset_folder_test.iterdir())[:100]


#initialising matrices
matrix_train = sp.load_npz(files_train[0]).toarray()
matrix_test = sp.load_npz(files_test[0]).toarray()

#creating matrices
for file_it in tqdm(files_train[1:]):
    matrix_train = np.concatenate((matrix_train, sp.load_npz(file_it).toarray()), axis=0 ) 

for file_it in files_test[1:]:
    matrix_test = np.concatenate((matrix_test, sp.load_npz(file_it).toarray()), axis=0 ) 

100%|██████████| 999/999 [00:44<00:00, 22.27it/s] 


In [80]:
testing_files = list(dataset_folder_test.iterdir())[100:1000]

test = sp.load_npz(testing_files[0]).toarray()

for file_it in tqdm(testing_files):
    test = np.concatenate((test, sp.load_npz(file_it).toarray()), axis=0 ) 


100%|██████████| 900/900 [00:36<00:00, 24.66it/s] 


In [81]:
test_X = test[:, :510]
test_y = one_hot( test[:, -1] )

In [82]:
X_train = matrix_train[:, :510]
y_train = one_hot( matrix_train[:, -1] )

X_test = matrix_test[:, :510]
y_test = one_hot( matrix_test[:, -1] )

In [125]:
import keras
from tensorflow.keras.layers import GlobalAveragePooling1D

inputs = Input(shape=(510,))

output_attention = keras.layers.MultiHeadAttention(num_heads=2, key_dim=510)(inputs, inputs)  #paper uses 1 head
print(output_attention.shape)

dense1 = Dense(units=1024, activation='relu')(output_attention)
dense2 = Dense(units=516, activation='relu')(dense1)
dense3 = Dense(units=256, activation='relu')(dense2)
dense4 = Dense(units=128, activation='relu')(dense3)
dense5 = Dense(units=34, activation='relu')(dense4)

outputs = Dense(units=34, activation='sigmoid')(dense5)  # For binary classification

model = Model(inputs=inputs, outputs=outputs)

model.compile(optimizer = SGD (learning_rate=0.01),
              loss='categorical_crossentropy', 
              metrics=['accuracy'])
model.summary()

(None, 510)


Model: "functional_48"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_39      │ (None, 510)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multi_head_attenti… │ (None, 510)       │  2,084,370 │ input_layer_39[0… │
│ (MultiHeadAttentio… │                   │            │ input_layer_39[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_194 (Dense)   │ (None, 1024)      │    523,264 │ multi_head_atten… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_195 (Dense)   │ (None, 516)       │    528,900 │ dense_194[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_196 (Dense)   │ (None, 256)       │    132,352 │ dense_195[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_197 (Dense)   │ (None, 128)       │     32,896 │ dense_196[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_198 (Dense)   │ (None, 34)        │      4,386 │ dense_197[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_199 (Dense)   │ (None, 34)        │      1,190 │ dense_198[0][0]   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 3,307,358 (12.62 MB)

 Trainable params: 3,307,358 (12.62 MB)

 Non-trainable params: 0 (0.00 B)

In [126]:
history = model.fit(X_train, y_train, epochs=5, batch_size=64, validation_data=(X_test, y_test))

Epoch 1/5


IndexError: Exception encountered when calling Softmax.call().

[1mtuple index out of range[0m

Arguments received by Softmax.call():
  • inputs=tf.Tensor(shape=(None, 2), dtype=float32)
  • mask=None

In [ ]:
test_loss, test_accuracy = model.evaluate(test_X, test_y)
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")

13727/13727 ━━━━━━━━━━━━━━━━━━━━ 69s 5ms/step - accuracy: 0.0495 - loss: 3.4368
Test Loss: 3.4363956451416016
Test Accuracy: 0.049554359167814255
